# Get Coast to Coast Assists and Buckets using PBP data


In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/frustration_coast/"

league = "NBA"
season_yr = "2022"
season_type = "Regular Season"
games_id = pbp_season(league=league,season_yr=season_yr ,season_type=season_type)
games_list = pbp_games(games_id)

In [ ]:
year = int(season_yr)
# player_dict = get_players(league = 'NBA', from_year = year, to_year = year)
player_dict = get_players_pbp(league = league)
team_dict = teams.get_teams() # Creating Team Dictionary
team_id = [x['id'] for x in team_dict ]

## Coast to Coast Assists

In [ ]:
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import Rebound
pos_store = []
Rebound_pID = []
Assist_pID = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= 5:
                if possession_event.previous_event.player1_id == possession_event.player2_id:
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        print ("Rebound Player: {0}".format(possession_event.previous_event.player1_id))
                        Assist_pID.append(possession_event.player2_id)
                        print ("Assist Player: {0}".format(possession_event.player2_id))
                        ii += 1
                        # print("ii = {0}".format(ii))
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

In [ ]:
## Code to figure out errors in player ID mapping
# Player_Name = []
# a = Assist_pID
# wrong_id = []
# Assist_pID1, Assists = np.unique(a, return_counts=True)  
# for pID in Assist_pID1:
#     abc = False
#     for player in player_dict:
#         if player['pID'] == pID:
#             abc= True
#             Player_Name.append(player['Name'])
#     if not abc:
#         wrong_id.append(pID)
# print(len(Assist_pID1))
# print(len(Player_Name))
# print(len(wrong_id))

In [ ]:
## Code to map pID to players using old format
# Player_Name = []
# a = Assist_pID
# Assist_pID1, Assists = np.unique(a, return_counts=True)  
# for pID in Assist_pID1:
#     Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
# for i in range(len(Player_Name)):
#     if not Player_Name[i]:
#         Player_Name[i] =['abc']
# Player_Name = list(itertools.chain(*Player_Name))

In [ ]:
Assist_pID1, Assists = np.unique(Assist_pID, return_counts=True)
Player_Name = np.array([player_dict[x] for x in Assist_pID1])

In [ ]:
data_ca = pd.DataFrame({'Assist_pID':Assist_pID1,'Player':Player_Name,'AST':Assists})
data_ca = data_ca.sort_values(by=['AST'],ascending=False)
data_ca.reset_index(drop=True,inplace=True)
data_ca.reset_index(drop=False,inplace=True)
data_ca["#"] = data_ca["index"] +1
data_ca = data_ca[["#","Player","AST"]]
data_ca = data_ca.head(10)

## Coast to Coast Buckets

In [ ]:
pos_store = []
Rebound_pID = []
FGM_pID = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and not possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= 8:
                if possession_event.previous_event.player1_id == possession_event.player1_id:
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        print ("Rebound Player: {0}".format(possession_event.previous_event.player1_id))
                        FGM_pID.append(possession_event.player1_id)
                        print ("FGM Player: {0}".format(possession_event.player1_id))
                        ii += 1
                        print("ii = {0}".format(ii))
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

In [ ]:
FGM_pID1, FGM = np.unique(FGM_pID, return_counts=True)  
Player_Name = np.array([player_dict[x] for x in FGM_pID1])

In [ ]:
data_cb = pd.DataFrame({'FGM_pID':FGM_pID1,'Player':Player_Name,'FGM':FGM})
data_cb = data_cb.sort_values(by=['FGM'],ascending=False)
data_cb.reset_index(drop=True,inplace=True)
data_cb.reset_index(drop=False,inplace=True)
data_cb["#"] = data_cb["index"] +1
data_cb = data_cb[["#","Player","FGM"]]
data_cb = data_cb.head(10)

In [ ]:
def plot_table(df,var,sort="Player",title=" ",footer=" ",col_width=15):
    fig = go.Figure(data=[go.Table(
        columnwidth=[5,40,col_width],
        header=dict(values=list(df.columns),
                    fill_color='blue',
                    align=['center','left','center'],
                    font=dict(color='snow',family="Arial Black, monospace", size=12),
                    line_color="snow"
                    ),
        cells=dict(values=[df["#"],df[sort],df[var]],
                fill_color='lavender',
                align=['center','left','center'],
                height=23,
                line_color="grey",#lightgrey",
                ),
                # height=25
        ),
    ])
    # fig.update_layout(title_text=title)
    fig.update_layout(title=dict(text=title,y=0.98,x=0.02,font=dict(size=14,family="Arial Black, monospace")))
    tab_width = 250 + col_width
    tab_height = 315
    fig.update_layout(width=tab_width,height=tab_height,margin=dict(t=25,b=1,l=1,r=1))
    fig.add_annotation(x=0.0, y=0.0,text="@SravanNBA",showarrow=False,xshift=1,yshift=1,font=dict(size=10))
    fig.add_annotation(x=1.0, y=0.0,text="Source: pbpstats",showarrow=False,xshift=1,yshift=1,font=dict(size=10))
    fig.add_annotation(x=0.1, y=0.0,text=footer,showarrow=False,xshift=0,yshift=15,font=dict(size=10))
    # fig.update_layout(autosize=True)
    fig.show()
    # fig.write_image(fig_DIR+f"{var}.png", scale=2)
    return fig

In [ ]:
# Players with the most Coast to Coast Buckets this season (Player grabs a defensive rebound and scores within 8 seconds)

In [ ]:
if league == "NBA":
    title = f"Coast to Coast Buckets {year}-{str(year+1)[2:]}"
elif league == "WNBA":
    title = f"Coast to Coast Buckets {year}"
footer = "Player scores within 8s of rebounding"
fig = plot_table(data_cb,"FGM",title=title,footer=footer,col_width=15)
fig.write_image(fig_DIR+f"{league}_coast_buckets_{year}.png", scale=3)

In [ ]:
if league == "NBA":
    title = f"Coast to Coast Assists {year}-{str(year+1)[2:]}"
elif league == "WNBA":
    title = f"Coast to Coast Assists {year}"
footer = "Player assists within 8s of rebounding"
fig = plot_table(data_ca,"AST",title=title,footer=footer,col_width=15)
fig.write_image(fig_DIR+f"{league}_coast_assists_{year}.png", scale=3)